In [1]:
import itertools

import torch

from torch_geometric.nn import GCNConv
from torch_geometric.nn.pool import global_mean_pool, global_max_pool
from torch.utils.data import DataLoader, random_split

In [2]:
from helpers import CVFConfigForGCNDataset
from metrics import CustomR2Score

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
color_mapping_categories = 15

# dataset = CVFConfigDataset(
#     "small_graph_test_config_rank_dataset.csv", "small_graph_edge_index.json", 4
# )

# dataset_n1 = CVFConfigForGCNDataset(
#     device,
#     "graph_1_config_rank_dataset.csv",
#     "graph_1_edge_index.json",
#     20
# )

# dataset_n4 = CVFConfigForGCNDataset(
#     device,
#     "graph_4_config_rank_dataset.csv",
#     "graph_4_edge_index.json",
#     color_mapping_categories
# )

# dataset_n5 = CVFConfigForGCNDataset(
#     device,
#     "graph_5_config_rank_dataset.csv",
#     "graph_5_edge_index.json",
#     color_mapping_categories
# )

# dataset_n6 = CVFConfigForGCNDataset(
#     device,
#     "graph_6_config_rank_dataset.csv",
#     "graph_6_edge_index.json",
#     color_mapping_categories,
# )

# dataset_n6b = CVFConfigForGCNDataset(
#     device,
#     "graph_6b_config_rank_dataset.csv",
#     "graph_6b_edge_index.json",
#     color_mapping_categories,
# )

# dataset_n7 = CVFConfigForGCNDataset(
#     device,
#     "graph_7_config_rank_dataset.csv",
#     "graph_7_edge_index.json",
#     color_mapping_categories,
# )

# # dataset_n8 = CVFConfigForGCNDataset(
# #     device,
# #     "graph_8_config_rank_dataset.csv",
# #     "graph_8_edge_index.json",
# #     color_mapping_categories,
# # )

# dataset_pl_n5 = CVFConfigForGCNDataset(
#     device,
#     "graph_powerlaw_cluster_graph_n5_config_rank_dataset.csv",
#     "graph_powerlaw_cluster_graph_n5_edge_index.json",
#     color_mapping_categories
# )

# dataset_pl_n6 = CVFConfigForGCNDataset(
#     device,
#     "graph_powerlaw_cluster_graph_n6_config_rank_dataset.csv",
#     "graph_powerlaw_cluster_graph_n6_edge_index.json",
#     color_mapping_categories
# )

# dataset_pl_n7 = CVFConfigForGCNDataset(
#     device,
#     "graph_powerlaw_cluster_graph_n7_config_rank_dataset.csv",
#     "graph_powerlaw_cluster_graph_n7_edge_index.json",
#     color_mapping_categories
# )

# dataset_pl_n8 = CVFConfigForGCNDataset(
#     device,
#     "graph_powerlaw_cluster_graph_n8_config_rank_dataset.csv",
#     "graph_powerlaw_cluster_graph_n8_edge_index.json",
#     color_mapping_categories
# )

# dataset_pl_n9 = CVFConfigForGCNDataset(
#     device,
#     "graph_powerlaw_cluster_graph_n9_config_rank_dataset.csv",
#     "graph_powerlaw_cluster_graph_n9_edge_index.json",
#     color_mapping_categories,
# )

# dataset_pl_n12 = CVFConfigForGCNDataset(
#     device,
#     "graph_powerlaw_cluster_graph_n12_config_rank_dataset.csv",
#     "graph_powerlaw_cluster_graph_n12_edge_index.json",
#     color_mapping_categories
# )

color_mapping_categories = 3
# dataset_implicit_n15 = CVFConfigForGCNDataset(
#     device,
#     "implicit_graph_n15_config_rank_dataset.csv",
#     "implicit_graph_n15_edge_index.json",
#     color_mapping_categories,
#     one_hot_encode=False
# )

# dataset_implicit_n10 = CVFConfigForGCNDataset(
#     device,
#     "implicit_graph_n10_config_rank_dataset.csv",
#     "implicit_graph_n10_edge_index.json",
#     color_mapping_categories,
#     one_hot_encode=True
# )

# dataset_implicit_n4 = CVFConfigForGCNDataset(
#     device,
#     "implicit_graph_n4_config_rank_dataset.csv",
#     "implicit_graph_n4_edge_index.json",
#     color_mapping_categories,
#     one_hot_encode=True
# )

dataset_implicit_n5 = CVFConfigForGCNDataset(
    device,
    "implicit_graph_n5_config_rank_dataset.csv",
    "implicit_graph_n5_edge_index.json",
    color_mapping_categories,
    one_hot_encode=True
)

batch_size = 64

dataset_coll = [dataset_implicit_n5]
train_dataloader_coll = []
test_dataloader_coll = []

for dataset in dataset_coll:
    train_size = int(0.75 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    train_dataloader_coll.append(train_loader)
    test_dataloader_coll.append(test_loader)

train_dataloader_coll_iter = [iter(i) for i in train_dataloader_coll]

In [5]:
def generate_batch():
    end_loop = [False for _ in range(len(train_dataloader_coll))]
    while not any(end_loop):
        for di, data_loader in enumerate(train_dataloader_coll_iter):
            if end_loop[di]:
                continue
            try:
                batch = next(data_loader)
            except StopIteration:
                end_loop[di] = True
                continue
            yield batch, di

In [6]:
# cnt = 0
# for i in generate_batch():
#     cnt += 1

# print(cnt)
print("Number of batches:", [len(i) for i in train_dataloader_coll])

Number of batches: [3]


In [7]:
import csv
csvfile = open("pl_n9__epoch_vs_acc.csv", "w")
writer = csv.DictWriter(csvfile, fieldnames=["epoch", "accuracy"])
writer.writeheader()


class VanillaGNN(torch.nn.Module):
    def __init__(self, dim_in, dim_h, dim_out):
        super().__init__()
        self.gcn1 = GCNConv(dim_in, dim_h)
        self.gcn2 = GCNConv(dim_h, dim_h)
        self.out = torch.nn.Linear(dim_h, dim_out)

    def forward(self, x, edge_index):
        h = self.gcn1(x, edge_index)
        h = torch.relu(h)
        h = self.gcn2(h, edge_index)
        h = torch.relu(h)
        h = self.out(h)
        h = torch.relu(h)
        h = global_mean_pool(h, torch.zeros(h.size(1)).to(device).long())
        return h

    def fit(self, epochs):
        # criterion = torch.nn.CrossEntropyLoss()
        # criterion = torch.nn.MSELoss()
        criterion_sum = torch.nn.MSELoss(reduction='sum')
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=5e-4)
        self.train()
        metric = CustomR2Score()
        # edge_index = dataset.edge_index.t().to(device)
        dataloaders = itertools.tee(generate_batch(), epochs)
        for epoch in range(1, epochs + 1):
            max_loss = -1
            total_loss = 0
            count = 0
            for batch, di in dataloaders[epoch-1]:
                x = batch[0].to(device)
                y = batch[1].to(device)
                y = y.unsqueeze(0).reshape(-1, 1, 1).float()
                optimizer.zero_grad()
                out = self(x, dataset_coll[di].edge_index)
                # print("output", out, "y", y)
                loss = criterion_sum(out, y)
                metric.update(y, out)
                # loss_sum = cr(out, y)
                avg_loss = loss/len(batch[1])
                if avg_loss > max_loss:
                    max_loss = avg_loss
                total_loss += loss
                count += len(batch[1])
                loss.backward()
                optimizer.step()

            if count > 0:
                accuracy = round(metric.compute()*100, 4)
                if accuracy < 0:
                    accuracy = 0
                writer.writerow({"epoch": epoch, "accuracy": accuracy})
                print("Epoch", epoch, "| Loss:", round((total_loss / count).item(), 4), "| Max Loss:", round(max_loss.item(), 4), "| Accuracy:", round(metric.compute()*100, 4))


In [8]:
gnn = VanillaGNN(color_mapping_categories, 64, 1).to(device)
print(gnn)

gnn.fit(epochs=50)

VanillaGNN(
  (gcn1): GCNConv(1, 64)
  (gcn2): GCNConv(64, 64)
  (out): Linear(in_features=64, out_features=1, bias=True)
)


RuntimeError: Class values must be smaller than num_classes.

In [9]:
# testing
torch.no_grad()
torch.set_printoptions(profile="full")

metric = CustomR2Score()
criterion = torch.nn.MSELoss(reduction="sum")
total_loss = 0

total_matched = 0

indx = 0
dataset = dataset_coll[indx]
test_loader = test_dataloader_coll[indx]
test_dataset = test_loader.dataset

# dataset = dataset_pl_n9
# train_size = int(0.9 * len(dataset))
# test_size = len(dataset) - train_size
# train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

r_acc = 0

for batch in test_loader:
    x = batch[0].to(device)
    y = batch[1].to(device)
    y = y.unsqueeze(0).reshape(-1, 1, 1).float()
    predicted = gnn(x, dataset.edge_index)
    # print(y.squeeze(-1).squeeze(-1)[:100], predicted.squeeze(-1).squeeze(-1)[:100])
    metric.update(y, predicted)
    loss = criterion(predicted, y)
    total_loss += loss
    predicted = torch.round(predicted)
    matched = (predicted == y).sum().item()
    total_matched += matched
    # print(y.squeeze(-1).squeeze(-1), predicted.squeeze(-1).squeeze(-1))

# print("Total matched", total_matched, "out of", len(test_dataset), "| Accuracy", round(total_matched/len(test_dataset) * 100, 4), "%")
print("Total matched", total_matched, "out of", len(test_dataset))
# print("Total Avg. Loss", total_loss/len(test_loader.dataset))
print("R-squared accuracy", round(metric.compute() * 100, 3), "%")

Total matched 0 out of 61
R-squared accuracy -1.913 %
